First of all, we need to install all the packages needed for this project

In [1]:
library("curatedTCGAData");
library("TCGAbiolinks");
library("TCGAutils");
library("SNFtool");
library("caret");
library("cluster"); #pam
library("mclustcomp");

Caricamento del pacchetto richiesto: MultiAssayExperiment

Caricamento del pacchetto richiesto: SummarizedExperiment

Caricamento del pacchetto richiesto: MatrixGenerics

Caricamento del pacchetto richiesto: matrixStats


Caricamento pacchetto: ‘MatrixGenerics’


I seguenti oggetti sono mascherati da ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMad

Download the **Prostate adenocarcinoma** dataset, considering three different omics data sources (**mRNA**, **miRNA** and **protein expression** data). The _TCGA_ code for the dataset is “PRAD”;
As above-mentioned, we will download **multi-omics** data from patients having prostate cancer. A multi-omics dataset is a dataset comprising multiple different biological data sources (i.e. messenger-RNA, micro-RNA, proteins, genomic mutations, etc), where each source represents a different data modality capturing the state of a specific biological layer in the cells. The underlying idea of using multiple data sources is that a biological system (normal or pathological) is the result of the interplay of multiple different molecules, thus considering many biological data sources at the same time gives us a more accurate picture of underlying processes.

The advent of the so-called “high-throughput technologies” enables the evaluation of:

- Genome: the complete genetic information of an organism (i.e. the sequence of nucleotides in the DNA)
- Transcriptome: set of all RNA transcripts (used also for all mRNA)
- Proteome: entire set of proteins etc.

in cell, tissue, or organism at a certain time. We can see that the suffix -ome it is used with the meaning of whole/entire. All omics data are high-dimensional and characterized by “small-n large-p” (i.e. few samples and a large number of features), which easily leads to the “curse of dimensionality” in machine learning applications.

We download a prostate cancer multi-omics dataset from The Cancer Genome Atlas (TCGA) program. TCGA is a genomics program that collected more than 11,000 cases across 33 tumor types, comprising multiple biological data sources (mRNA expression, miRNA expression, copy number values, DNA methylation, protein expression). In particular, we exploit the package “curatedTCGAData” to download the following data views:

- mRNA data
- miRNA data
- protein data

The complete list of data types is available in the help page of the function ?curatedTCGAData

In [2]:
# Download prostate cancer multi-omics dataset
# Note that RPPA stands for Reverse-phase protein array and it is 
# the technology used to obtain proteomic data.
# Use ?curatedTCGAData to access the help page of the function and see
# the table with all the available data types (not all types are available for
# each cancer).
assays <- c("miRNASeqGene", "RNASeq2Gene", "RPPAArray");
mo <- curatedTCGAData(diseaseCode = "PRAD", 
                        assays = assays, 
                        version = "2.0.1", dry.run = FALSE);
mo

snapshotDate(): 2022-10-31

Working on: PRAD_miRNASeqGene-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Working on: PRAD_RNASeq2Gene-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Working on: PRAD_RPPAArray-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Working on: PRAD_colData-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Working on: PRAD_metadata-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Working on: PRAD_sampleMap-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

harmonizing input:
  removing 5189 sampleMap rows not in names(experiments)



A MultiAssayExperiment object of 3 listed
 experiments with user-defined names and respective classes.
 Containing an ExperimentList class object of length 3:
 [1] PRAD_miRNASeqGene-20160128: SummarizedExperiment with 1046 rows and 547 columns
 [2] PRAD_RNASeq2Gene-20160128: SummarizedExperiment with 20501 rows and 550 columns
 [3] PRAD_RPPAArray-20160128: SummarizedExperiment with 195 rows and 352 columns
Functionality:
 experiments() - obtain the ExperimentList instance
 colData() - the primary/phenotype DataFrame
 sampleMap() - the sample coordination DataFrame
 `$`, `[`, `[[` - extract colData columns, subset, or experiment
 *Format() - convert into a long or wide DataFrame
 assays() - convert ExperimentList to a SimpleList of matrices
 exportClass() - save data to flat files

In [3]:
sampleMap(mo)

DataFrame with 1449 rows and 3 columns
                         assay      primary                colname
                      <factor>  <character>            <character>
1    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VL TCGA-2A-A8VL-01A-21R..
2    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VO TCGA-2A-A8VO-01A-11R..
3    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VT TCGA-2A-A8VT-01A-11R..
4    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VV TCGA-2A-A8VV-01A-11R..
5    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VX TCGA-2A-A8VX-01A-11R..
...                        ...          ...                    ...
1445   PRAD_RPPAArray-20160128 TCGA-ZG-A9LZ TCGA-ZG-A9LZ-01A-21-..
1446   PRAD_RPPAArray-20160128 TCGA-ZG-A9M4 TCGA-ZG-A9M4-01A-21-..
1447   PRAD_RPPAArray-20160128 TCGA-ZG-A9MC TCGA-ZG-A9MC-01A-21-..
1448   PRAD_RPPAArray-20160128 TCGA-ZG-A9N3 TCGA-ZG-A9N3-01A-22-..
1449   PRAD_RPPAArray-20160128 TCGA-ZG-A9NI TCGA-ZG-A9NI-01A-21-..

In [4]:
mo <- mo[, , paste0("PRAD", "_", assays, "-20160128")];
sampleMap(mo)

DataFrame with 1449 rows and 3 columns
                         assay      primary                colname
                      <factor>  <character>            <character>
1    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VL TCGA-2A-A8VL-01A-21R..
2    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VO TCGA-2A-A8VO-01A-11R..
3    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VT TCGA-2A-A8VT-01A-11R..
4    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VV TCGA-2A-A8VV-01A-11R..
5    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VX TCGA-2A-A8VX-01A-11R..
...                        ...          ...                    ...
1445   PRAD_RPPAArray-20160128 TCGA-ZG-A9LZ TCGA-ZG-A9LZ-01A-21-..
1446   PRAD_RPPAArray-20160128 TCGA-ZG-A9M4 TCGA-ZG-A9M4-01A-21-..
1447   PRAD_RPPAArray-20160128 TCGA-ZG-A9MC TCGA-ZG-A9MC-01A-21-..
1448   PRAD_RPPAArray-20160128 TCGA-ZG-A9N3 TCGA-ZG-A9N3-01A-22-..
1449   PRAD_RPPAArray-20160128 TCGA-ZG-A9NI TCGA-ZG-A9NI-01A-21-..

As we can see, we obtain a MultiAssayExperiment object, which, in its essence, is a data structure designed to store and coordinately analyze multi-omics experiments. In a way, it can be seen as an extension of the ExpressionSet class for more than one omic. The three main components of this data structure are:
- colData: it contains a dataframe having for each sample the corresponding phenotipic characteristics (in our case mainly clinical data) - access colData()
- ExperimentList: a list with the considered experiments (i.e. data modalities acquired with a specific technology). Element of the list are usually matrices or dataframes - access experiments()
- sampleMap: it is a map that connects all the considered elements.- access sampleMap()

Moreover, a function is provided to build MultiAssayExperiment objects for your own data and also subsetting operations for coordinated data selection among views.

Pre-process the dataset following the same steps we used during lessons. During the filtering by variance, select the first $100$ features having highest variance from each data source;
To work with data coming from TCGA, it is important to understand the structure of the barcode associated to each sample. Each sample/patient is identified by a barcode with a specific structure: in pratice, the first 12 characters of the barcode identify a specific individual, while the other parts give us indications about the type of sample (i.e. primary, metastatic, solid, blood derived, etc), the type of genomic material extracted (i.e. DNA, RNA) and other information related to technical replicates (i.e. repeated measurements from the same sample).

In [5]:
# Consider only primary solid tumors:
primary <- TCGAutils::TCGAsampleSelect(colnames(mo), c("01"));
primary

Warning message in .checkBarcodes(barcodes, check.sample = TRUE):
“Inconsistent barcode lengths: 28, 27”
Warning message in .checkBarcodes(barcodes):
“Inconsistent barcode lengths: 28, 27”


LogicalList of length 3
[["PRAD_miRNASeqGene-20160128"]] 01=TRUE 01=TRUE 01=TRUE ... 01=TRUE 01=TRUE
[["PRAD_RNASeq2Gene-20160128"]] 01=TRUE 01=TRUE 01=TRUE ... 01=TRUE 01=TRUE
[["PRAD_RPPAArray-20160128"]] 01=TRUE 01=TRUE 01=TRUE ... 01=TRUE 01=TRUE

In [6]:
# The execution of the precedent cell raises a warnin due to the fact
# that the barcode associated with the RPPAArray is composed by 27
# characters while the others are composed by 28 characters.
print(colnames(mo)[1]);
print(colnames(mo)[2]);
print(colnames(mo)[3]);

CharacterList of length 1
[["PRAD_miRNASeqGene-20160128"]] TCGA-2A-A8VL-01A-21R-A37H-13 ...
CharacterList of length 1
[["PRAD_RNASeq2Gene-20160128"]] TCGA-2A-A8VL-01A-21R-A37L-07 ...
CharacterList of length 1
[["PRAD_RPPAArray-20160128"]] TCGA-2A-A8VL-01A-11-A43M-20 ...


In [7]:
mo <- mo[, primary, ];

harmonizing input:
  removing 106 sampleMap rows with 'colname' not in colnames of experiments



In [8]:
sampleMap(mo)

DataFrame with 1343 rows and 3 columns
                         assay      primary                colname
                      <factor>  <character>            <character>
1    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VL TCGA-2A-A8VL-01A-21R..
2    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VO TCGA-2A-A8VO-01A-11R..
3    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VT TCGA-2A-A8VT-01A-11R..
4    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VV TCGA-2A-A8VV-01A-11R..
5    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VX TCGA-2A-A8VX-01A-11R..
...                        ...          ...                    ...
1339   PRAD_RPPAArray-20160128 TCGA-ZG-A9LZ TCGA-ZG-A9LZ-01A-21-..
1340   PRAD_RPPAArray-20160128 TCGA-ZG-A9M4 TCGA-ZG-A9M4-01A-21-..
1341   PRAD_RPPAArray-20160128 TCGA-ZG-A9MC TCGA-ZG-A9MC-01A-21-..
1342   PRAD_RPPAArray-20160128 TCGA-ZG-A9N3 TCGA-ZG-A9N3-01A-22-..
1343   PRAD_RPPAArray-20160128 TCGA-ZG-A9NI TCGA-ZG-A9NI-01A-21-..

In [10]:
# Check for replicates (anyReplicated checks the so called biological or 
# primary unit in the sampleMap of the MultiAssayExperiment object, that
# corresponds to the first 12 characters of the barcodes for TCGA data):
check_rep <- anyReplicated(mo);
print(check_rep);

 PRAD_RNASeq2Gene-20160128 PRAD_miRNASeqGene-20160128 
                     FALSE                      FALSE 
   PRAD_RPPAArray-20160128 
                     FALSE 


Then, other additional pre-processing steps are performed:

- Remove FFPE (formalin-fixed, paraffin-embedded) samples. Broadly speaking, after a biopsy is performed we need to store and preserve the sample. Two major tissue preparation methods are generally used: (1) FFPE, (2) freezing the sample (see this link to know more about the two methodologies). DNA and RNA molecules are preserved better if the tissue is frozen, thus we will remove samples preserved using FFPE technique;
- Restrict samples to the ones having all the considered omics and extract the set of omics (one matrix for each omic) in a list;
- Transpose each matrix to have samples in the rows and features in columns.


In [11]:
# The information regarding if the sample is FFPE is stored in the clinical data,
# which are accessible using colData(). 
no_ffpe <- which(as.data.frame(colData(mo))$patient.samples.sample.is_ffpe == "no");
mo <- mo[, no_ffpe, ];
sampleMap(mo)

DataFrame with 1343 rows and 3 columns
                         assay      primary                colname
                      <factor>  <character>            <character>
1    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VL TCGA-2A-A8VL-01A-21R..
2    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VO TCGA-2A-A8VO-01A-11R..
3    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VT TCGA-2A-A8VT-01A-11R..
4    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8VV TCGA-2A-A8VV-01A-11R..
5    PRAD_RNASeq2Gene-20160128 TCGA-2A-A8W1 TCGA-2A-A8W1-01A-11R..
...                        ...          ...                    ...
1339   PRAD_RPPAArray-20160128 TCGA-ZG-A9LB TCGA-ZG-A9LB-01A-21-..
1340   PRAD_RPPAArray-20160128 TCGA-TP-A8TV TCGA-TP-A8TV-01A-21-..
1341   PRAD_RPPAArray-20160128 TCGA-V1-A9OX TCGA-V1-A9OX-01A-21-..
1342   PRAD_RPPAArray-20160128 TCGA-YL-A9WX TCGA-YL-A9WX-01A-11-..
1343   PRAD_RPPAArray-20160128 TCGA-ZG-A9L9 TCGA-ZG-A9L9-01A-21-..

In [12]:
# Obtain samples having all the considered omics:
complete <- intersectColumns(mo);

# Extract assays in list:
complete <- assays(complete);

# Obtain matrices samples x features:
complete <- lapply(complete, FUN=t)

Download the disease subtypes (column “Subtype\_Integrative” is the one containing the iCluster molecular subtypes). Note that not all subtypes are available for the set of samples having all the considered omics data sources, thus you need to retain from the multi-omics dataset only samples having an associated subtype;

Check that patients in multi-omics dataset and subtypes are in the same order;

Integrate the data using Similarity Network Fusion [3] with the scaled exponential euclidean distance;

Try to integrate the similarity matrices from each data source (computed by scaled exponential euclidean distance) using a simple average of the matrices. This can be considered as a trivial multi-omics data integration strategy;

[GROUP] Integrate the dataset using another data fusion method called NEMO [4] to obtain an integrated similarity matrix. NEMO implementation is available on github ([https://github.com/Shamir-Lab/NEMO]).(https://github.com/Shamir-Lab/NEMO);